In [1]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg
from mv_wg_avg import exp_weight_avg

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID_BUREAU'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV']

2018-08-10 11:45:19,664 logger 19 [INFO]    [logger_func] start 
2018-08-10 11:45:19,736 logger 19 [INFO]    [logger_func] start 
2018-08-10 11:45:19,736 logger 19 [INFO]    [logger_func] start 


In [2]:
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
# app = pd.read_csv('../data/application_train_test_after.csv')
prev = pd.read_csv('../data/previous_cleansing.csv')
bureau = pd.read_csv('../data/bureau_cleansing.csv')

balance = pd.read_csv('../data/bureau_balance.csv')
balance['STATUS'] =  balance['STATUS'].map(lambda x: '-1' if x=='C' or x=='X' else x).astype('int')
b_status = balance.groupby(b_id)['STATUS'].max()
b_status = b_status.reset_index().query('STATUS>=0')
bureau = bureau.merge(b_status, on=b_id, how='left')

# pos['DPD_CODE'] = pos['SK_DPD'].map(lambda x: 'DPD' if x>0 else 'NAN')
# pos['DPD_DEF_CODE'] = pos['SK_DPD_DEF'].map(lambda x: 'DPD_DEF' if x>0 else 'NAN')
# ccb['DPD_CODE'] = ccb['SK_DPD'].map(lambda x: 'DPD' if x>0 else 'NAN')
# ccb['DPD_DEF_CODE'] = ccb['SK_DPD_DEF'].map(lambda x: 'DPD_DEF' if x>0 else 'NAN')

In [24]:
' 延滞ID '
dpd_user_1 = list(ccb.query("SK_DPD > 0")[unique_id].drop_duplicates().values)
dpd_user_2 = list(ccb.query("NAME_CONTRACT_STATUS == 'Demand'")[unique_id].drop_duplicates().values)
dpd_user = list(set(dpd_user_1 + dpd_user_2))
len(dpd_user)
# ccb.query("SK_DPD > 0")['NAME_CONTRACT_STATUS'].drop_duplicates()
# ccb['SK_DPD'].drop_duplicates()
# ccb = pd.read_csv('../data/credit_card_balance.csv')
# ins = pd.read_csv('../data/installments_payments.csv')
# pos = pd.read_csv('../data/POS_CASH_balance.csv')

20662

In [6]:
for col in prev.columns:
    bef_len = len(prev[col])
    aft_len = len(prev[col].dropna())
    print(f'{col}: {aft_len}/{bef_len}')
# prev.head(50)

SK_ID_PREV: 1670214/1670214
SK_ID_CURR: 1670214/1670214
NAME_CONTRACT_TYPE: 1670214/1670214
AMT_ANNUITY: 1296342/1670214
AMT_APPLICATION: 1277812/1670214
AMT_CREDIT: 1333445/1670214
AMT_DOWN_PAYMENT: 404516/1670214
AMT_GOODS_PRICE: 1277830/1670214
WEEKDAY_APPR_PROCESS_START: 1670214/1670214
HOUR_APPR_PROCESS_START: 1670214/1670214
FLAG_LAST_APPL_PER_CONTRACT: 1670214/1670214
NFLAG_LAST_APPL_IN_DAY: 1670214/1670214
RATE_DOWN_PAYMENT: 404516/1670214
RATE_INTEREST_PRIMARY: 5951/1670214
RATE_INTEREST_PRIVILEGED: 5951/1670214
NAME_CASH_LOAN_PURPOSE: 1670214/1670214
NAME_CONTRACT_STATUS: 1670214/1670214
DAYS_DECISION: 1670214/1670214
NAME_PAYMENT_TYPE: 1670214/1670214
CODE_REJECT_REASON: 1670214/1670214
NAME_TYPE_SUITE: 849809/1670214
NAME_CLIENT_TYPE: 1670214/1670214
NAME_GOODS_CATEGORY: 1670214/1670214
NAME_PORTFOLIO: 1670214/1670214
NAME_PRODUCT_TYPE: 1670214/1670214
CHANNEL_TYPE: 1670214/1670214
SELLERPLACE_AREA: 907539/1670214
NAME_SELLER_INDUSTRY: 1670214/1670214
CNT_PAYMENT: 1297984/1

In [3]:
prev.head(10)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,TARGET,valid_no,is_train,is_test
0,2462805,299809,Cash loans,9842.400,90000.000,90000.0,NaN,90000.000,THURSDAY,13,Y,1.0,NaN,NaN,NaN,XNA,Approved,-75.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,-75.0,-74.0,-63.0,-63.0,-63.0,0.0,0,3.0,1,0
1,1626275,426902,Cash loans,5246.010,45000.000,47970.0,NaN,45000.000,THURSDAY,15,Y,1.0,NaN,NaN,NaN,XNA,Approved,-74.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,-74.0,-73.0,-62.0,-62.0,-62.0,1.0,0,3.0,1,0
2,1983905,157753,Cash loans,8610.120,112500.000,127350.0,NaN,112500.000,THURSDAY,14,Y,1.0,NaN,NaN,NaN,XNA,Approved,-74.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,24.0,high,Cash Street: high,-74.0,-73.0,-50.0,-50.0,-50.0,1.0,0,4.0,1,0
3,2462807,343731,Cash loans,17887.500,225000.000,225000.0,NaN,225000.000,THURSDAY,12,Y,1.0,NaN,NaN,NaN,XNA,Approved,-77.0,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,18.0,middle,Cash Street: middle,-77.0,-76.0,-59.0,-59.0,-59.0,0.0,0,2.0,1,0
4,2006470,429750,Cash loans,19681.965,270000.000,299938.5,NaN,270000.000,THURSDAY,18,Y,1.0,NaN,NaN,NaN,XNA,Approved,-80.0,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,24.0,middle,Cash Street: middle,-84.0,-83.0,-60.0,-64.0,-64.0,1.0,-1,-1.0,0,1
5,1852667,133853,Consumer loans,4403.565,43606.350,43389.0,4361.850,43606.350,THURSDAY,12,Y,1.0,0.099484,0.142440,0.637949,XAP,Approved,-15.0,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,30.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-14.0,-3.0,-11.0,-11.0,0.0,0,4.0,1,0
6,1663907,107678,Consumer loans,2684.520,29551.050,26451.0,5908.050,29551.050,SUNDAY,10,Y,1.0,0.198844,0.160716,0.715645,XAP,Approved,-13.0,Cash through the bank,XAP,Unaccompanied,Refreshed,Mobile,POS,XNA,Country-wide,92.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-12.0,-1.0,-10.0,-10.0,0.0,0,4.0,1,0
7,1013587,347427,Consumer loans,6155.460,75815.685,60651.0,15164.685,75815.685,TUESDAY,16,Y,1.0,0.217840,0.160716,0.715645,XAP,Approved,-8.0,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,25.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-7.0,3.0,-4.0,-4.0,0.0,0,1.0,1,0
8,1818171,142422,Consumer loans,6800.220,66916.800,67375.8,6696.000,66916.800,THURSDAY,16,Y,1.0,0.098452,0.193330,0.852537,XAP,Approved,-3.0,Cash through the bank,XAP,Unaccompanied,Repeater,Mobile,POS,XNA,Country-wide,93.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-1.0,9.0,0.0,0.0,0.0,0,4.0,1,0
9,2798042,142422,Consumer loans,3982.905,43606.350,39244.5,4361.850,43606.350,THURSDAY,18,Y,1.0,0.108939,0.142440,0.637949,XAP,Approved,-19.0,Cash through the bank,XAP,"Spouse, partner",Repeater,Mobile,POS,XNA,Country-wide,50.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-18.0,-7.0,-18.0,-18.0,0.0,0,4.0,1,0


In [203]:
cat = get_categorical_features(data=prev, ignore=ignore_features)
for c in cat:
    prev[c].drop_duplicates()

In [202]:
# tmp = prev.copy()
tmp = prev.query('TARGET!=-1')
tmp_avg = tmp[[unique_id, target, 'NAME_TYPE_SUITE']].fillna('NA').groupby('NAME_TYPE_SUITE')[target].mean().to_frame()
tmp_size = tmp[[unique_id, target, 'NAME_TYPE_SUITE']].fillna('NA').groupby('NAME_TYPE_SUITE').size().to_frame()
result = tmp_avg.join(tmp_size)
print(result)
sys.exit()
print(tmp['NAME_TYPE_SUITE'].drop_duplicates())
# data = prev
data['NAME_TYPE_SUITE'] = data['NAME_TYPE_SUITE'].map({'Children': 'middle_row_risk', 'Family': 'middle_row_risk', 'Unaccompanied': 'middle_risk', 'Group of people': 'hight_risk', 'Other_A': 'high_risk', 'Other_B': 'high_risk', 'Spouse, partner': 'row_risk'
})
# data['NAME_TYPE_SUITE'].drop_duplicates()

                   TARGET       0
NAME_TYPE_SUITE                  
Children         0.063565   27106
Family           0.074742  181880
Group of people  0.088647    1929
NA               0.093207  694672
Other_A          0.087084    7820
Other_B          0.084920   15073
Spouse, partner  0.082242   57586
Unaccompanied    0.082844  427635


SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [87]:
bureau['OVERDUE_CODE'] = bureau['AMT_CREDIT_MAX_OVERDUE'].map(lambda x: 'OVERDUE' if x>0 else 'NAN')

In [132]:
# bureau.sort_values(by=[unique_id, b_id, 'DAYS_CREDIT'], ascending=False, inplace=True)
# bureau = row_number(bureau, unique_id)
# prev.sort_values(by=[unique_id, p_id, 'DAYS_DECISION'], ascending=False, inplace=True)
# prev = row_number(prev, unique_id)

pos.sort_values(by=[unique_id, p_id, 'MONTHS_BALANCE'], ascending=False, inplace=True)
pos = row_number(pos, unique_id)

ccb.sort_values(by=[unique_id, p_id, 'MONTHS_BALANCE'], ascending=False, inplace=True)
ccb = row_number(ccb, unique_id)

In [181]:
result = []
freq = 3
term = -24
suffix = f'_latest{freq}_term{term}'
for i, df in enumerate([app, bureau, prev, ccb, pos]):
    if i==0:
        prefix = 'a_'
    elif i==1:
        prefix = 'b_'
        df = df.query(f'row_no<={freq}')
    elif i==2:
        prefix = 'p_'
        df = df.query(f'row_no<={freq}')
    elif i==3:
        prefix = 'cb_'
        df = df.query(f'MONTHS_BALANCE>={term}')
    elif i==4:
        prefix = 'pos_'
        df = df.query(f'MONTHS_BALANCE>={term}')
        
    df = df.query('TARGET!=-1')
    cat_cols = get_categorical_features(data=df, ignore=ignore_features)
    
    for c in cat_cols:
        tmp = df[[unique_id, target, c]].drop_duplicates()
        target_avg = tmp.groupby(c)[target].mean().to_frame()
    #     target_size = tmp.groupby(c)[target].size().to_frame().rename(columns={target:'cnt'})
    #     result = target_avg.join(target_size)
        ' all '
#         risk = target_avg.to_dict()[target]
        ' high_risk '
#         risk = target_avg.query('TARGET>=0.115').to_dict()[target]
        ' middle_risk '
#         risk = target_avg.query('TARGET>0.095').query('TARGET<0.115').to_dict()[target]
        ' row_risk '
        risk = target_avg.query('TARGET<=0.075').to_dict()[target]
        tmp_result = df[[unique_id, c]].set_index(unique_id)[c].map(risk).reset_index()
        tmp_result = tmp_result.groupby(unique_id, as_index=False)[c].mean()
        tmp_result.columns = [unique_id] + [f'{prefix}{col}' for col in tmp_result.columns if col not in ignore_features]
        if len(result)==0:
            result = base.merge(tmp_result, on=unique_id, how='left')
        else:
            result = result.merge(tmp_result, on=unique_id, how='left')
display(result.head())
# result.to_csv(f'../data/positive_table{suffix}.csv', index=False)
# result.to_csv(f'../data/high_risk_table{suffix}.csv', index=False)
# result.to_csv(f'../data/middle_risk_table{suffix}.csv', index=False)
result.to_csv(f'../data/row_risk_table{suffix}.csv', index=False)

,SK_ID_CURR,a_CODE_GENDER,a_EMERGENCYSTATE_MODE,a_FLAG_OWN_CAR,a_FLAG_OWN_REALTY,a_FONDKAPREMONT_MODE,a_HOUSETYPE_MODE,a_NAME_CONTRACT_TYPE,a_NAME_EDUCATION_TYPE,a_NAME_FAMILY_STATUS,a_NAME_HOUSING_TYPE,a_NAME_INCOME_TYPE,a_NAME_TYPE_SUITE,a_OCCUPATION_TYPE,a_ORGANIZATION_TYPE,a_WALLSMATERIAL_MODE,a_WEEKDAY_APPR_PROCESS_START,b_CREDIT_ACTIVE,b_CREDIT_CURRENCY,b_CREDIT_TYPE,b_OVERDUE_CODE,p_NAME_CONTRACT_TYPE,p_WEEKDAY_APPR_PROCESS_START,p_FLAG_LAST_APPL_PER_CONTRACT,p_NAME_CASH_LOAN_PURPOSE,p_NAME_CONTRACT_STATUS,p_NAME_PAYMENT_TYPE,p_CODE_REJECT_REASON,p_NAME_TYPE_SUITE,p_NAME_CLIENT_TYPE,p_NAME_GOODS_CATEGORY,p_NAME_PORTFOLIO,p_NAME_PRODUCT_TYPE,p_CHANNEL_TYPE,p_NAME_SELLER_INDUSTRY,p_NAME_YIELD_GROUP,p_PRODUCT_COMBINATION,cb_NAME_CONTRACT_STATUS,cb_DPD_CODE,cb_DPD_DEF_CODE,pos_NAME_CONTRACT_STATUS,pos_DPD_CODE,pos_DPD_DEF_CODE
0,100002,NaN,0.069649,NaN,NaN,0.069782,0.069434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.074057,NaN,0.069323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071829,NaN,0.069262,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.069993,0.069649,NaN,NaN,0.069782,0.069434,NaN,0.053551,NaN,NaN,0.05755,0.074946,0.06304,0.059148,0.070247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071895,0.066328,0.062896,NaN,NaN,0.071829,0.058354,0.069262,0.060368,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,NaN,NaN,0.072437,NaN,NaN,NaN,0.054783,NaN,NaN,NaN,NaN,NaN,NaN,0.069781,NaN,NaN,0.069323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.069993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066133,NaN,NaN,0.071829,0.062216,NaN,0.060502,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.06304,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.073857,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
rank = 'high'
freq = 3
term = -24
risk = pd.read_csv(f'../data/{rank}_risk_table_latest{freq}_term{term}.csv')
risk = pd.read_csv(f'../data/positive_table_latest{freq}_term{term}.csv')
risk = pd.read_csv(f'../data/risk_all_yoko_latest{freq}_term{term}.csv')
# risk = pd.read_csv(f'../data/risk_all_yoko.csv')

tmp = risk.set_index(unique_id).fillna(0).sum(axis=1).values
np.save(arr=tmp, file=f'../category_risk_{rank}_positive_freq{freq}_term{term}_sum@')
# np.save(arr=tmp, file=f'../category_risk_all_positive_freq{freq}_term{term}_sum@')
# np.save(arr=tmp, file=f'../category_risk_all_positive_sum@')

tmp = risk.set_index(unique_id).fillna(0).std(axis=1).values
np.save(arr=tmp, file=f'../category_risk_{rank}_positive_freq{freq}_term{term}_std@')
# np.save(arr=tmp, file=f'../category_risk_all_positive_freq{freq}_term{term}_std@')
# np.save(arr=tmp, file=f'../category_risk_all_positive_std@')

In [182]:
def df_rename(df, rank):
    for col in df.columns:
        if col not in ignore_features:
            df.rename(columns={col:f'{rank}_{col}'}, inplace=True)
        
freq = 3
term = -24
high = pd.read_csv(f'../data/high_risk_table_latest{freq}_term{term}.csv', index_col=unique_id)
middle = pd.read_csv(f'../data/middle_risk_table_latest{freq}_term{term}.csv', index_col=unique_id)
row = pd.read_csv(f'../data/row_risk_table_latest{freq}_term{term}.csv', index_col=unique_id)
# high = pd.read_csv(f'../data/high_risk_table.csv', index_col=unique_id)
# middle = pd.read_csv(f'../data/middle_risk_table.csv', index_col=unique_id)
# row = pd.read_csv(f'../data/row_risk_table.csv', index_col=unique_id)

for rank in ['high', 'middle', 'row']:
    if rank.count('high'): df_rename(high, rank)
    elif rank.count('middle'): df_rename(middle, rank)
    elif rank.count('row'): df_rename(row, rank)
        
df_rank = high.join(middle).join(row)
df_rank.to_csv(f'../data/risk_all_yoko_latest{freq}_term{term}.csv', index=True)
# df_rank.to_csv(f'../data/risk_all_yoko.csv', index=True)

In [188]:
' リスクの種類毎にvariety数をカウントし、その数でターゲットエンコーディング '
freq = 3
term = -24
df = pd.read_csv(f'../data/risk_all_yoko_latest{freq}_term{term}.csv', index_col=unique_id)
df[df>0] = 1
df.fillna(0).head()

rank_list = ['high', 'middle', 'row']
    
high_cols = [col for col in df.columns if col.count('high')]
middle_cols = [col for col in df.columns if col.count('middle')]
row_cols = [col for col in df.columns if col.count('row')]

df['high_variety@'] = df[high_cols].sum(axis=1)
df['middle_variety@'] = df[middle_cols].sum(axis=1)
df['row_variety@'] = df[row_cols].sum(axis=1)
variety_cols = [col for col in df.columns if col.count('variety')]
tmp = df[variety_cols].astype('str').apply(lambda x:x[0] + x[1] + x[2])
tmp.head()

KeyError: (0, 'occurred at index high_variety@')

In [4]:
abp['CNT_ANNUITY'] = abp['AMT_CREDIT'] / abp['AMT_ANNUITY']
abp['CREDIT_TERM'] = abp['DAYS_LAST_DUE'].fillna(3) - abp['DAYS_DECISION']
abp['AMT_ANNUITY'].fillna(abp['AMT_CREDIT']/abp['CREDIT_TERM'], inplace=True)
abp['SK_ID'] = np.arange(len(abp))

apr = abp.query("NAME_CONTRACT_STATUS=='Approved'")

In [15]:
' 申請したクレジット順に行番号を付与し、回数で切って特徴量を作成する '
amt_list = ['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY']
monthly = apr.groupby([unique_id, 'DAYS_DECISION'])[amt_list].sum().reset_index()
df = monthly
df.sort_values(by=[unique_id, 'DAYS_DECISION'], ascending=False, inplace=True)
df = row_number(df=df, level=unique_id)
df = df.merge(app[[unique_id, 'AMT_INCOME_TOTAL', 'DAYS_BIRTH_log10']], on=unique_id, how='inner')
for amt in amt_list:
    df = division_feature(df=df, first=amt, second='AMT_INCOME_TOTAL')
    df = division_feature(df=df, first=amt, second='DAYS_BIRTH_log10')
df.head(10)

,SK_ID_CURR,DAYS_DECISION,AMT_CREDIT,AMT_ANNUITY,CNT_ANNUITY,row_no,AMT_INCOME_TOTAL,DAYS_BIRTH_log10,AMT_CREDIT_div_AMT_INCOME_TOTAL@,AMT_CREDIT_div_DAYS_BIRTH_log10@,AMT_ANNUITY_div_AMT_INCOME_TOTAL@,AMT_ANNUITY_div_DAYS_BIRTH_log10@,AMT_INCOME_TOTAL_div_CNT_ANNUITY@,CNT_ANNUITY_div_DAYS_BIRTH_log10@
0,456255,0.0,675000.00,49117.500,13.742556,1,157500.0,2.447158,4.285714,275830.163543,0.311857,20071.241567,11460.750000,5.615721
1,456255,-15.0,93298.50,11314.170,8.246164,2,157500.0,2.447158,0.592371,38125.245205,0.071836,4623.391647,19099.790190,3.369690
2,456255,-16.0,58545.00,11514.555,5.084434,3,157500.0,2.447158,0.371714,23923.669518,0.073108,4705.276428,30976.896618,2.077689
3,456255,-21.0,254340.00,16400.610,15.507960,4,157500.0,2.447158,1.614857,103932.805623,0.104131,6701.900650,10156.074841,6.337130
4,456255,-23.0,491580.00,28873.170,17.025495,5,157500.0,2.447158,3.121143,200877.913769,0.183322,11798.653634,9250.832570,6.957252
5,456255,-26.0,910208.43,44946.990,20.250709,6,157500.0,2.447158,5.779101,371945.096452,0.285378,18367.015707,7777.505340,8.275195
6,456255,-33.0,112815.00,11090.835,10.171912,7,157500.0,2.447158,0.716286,46100.414667,0.070418,4532.128640,15483.814320,4.156623
7,456254,0.0,370107.00,20205.000,18.317595,1,171000.0,2.646404,2.164368,139852.810942,0.118158,7634.889492,9335.286822,6.921693
8,456254,-9.0,247423.50,19065.825,12.977330,2,171000.0,2.646404,1.446921,93494.238067,0.111496,7204.427961,13176.824655,4.903760
9,456254,-10.0,21456.00,2296.440,9.343157,3,171000.0,2.646404,0.125474,8107.606480,0.013429,867.758754,18302.164430,3.530511


In [17]:
num_list = [col for col in df.columns if col.count('@')]
num_list = num_list + ['DAYS_DECISION'] + amt_list
num_list

['AMT_CREDIT_div_AMT_INCOME_TOTAL@',
 'AMT_CREDIT_div_DAYS_BIRTH_log10@',
 'AMT_ANNUITY_div_AMT_INCOME_TOTAL@',
 'AMT_ANNUITY_div_DAYS_BIRTH_log10@',
 'AMT_INCOME_TOTAL_div_CNT_ANNUITY@',
 'CNT_ANNUITY_div_DAYS_BIRTH_log10@',
 'DAYS_DECISION',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'CNT_ANNUITY']

In [47]:
' EMPLOYEDとREGISTRAION前後で集計を行い、特徴を作成する '
num_list = ['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY']
emp_bef = apr.query('DAYS_DECISION < DAYS_EMPLOYED')[num_list+[unique_id]]
emp_aft = apr.query('DAYS_DECISION >= DAYS_EMPLOYED')[num_list+[unique_id]]
reg_bef = apr.query('DAYS_DECISION < DAYS_REGISTRATION')[num_list+[unique_id]]
reg_aft = apr.query('DAYS_DECISION >= DAYS_REGISTRATION')[num_list+[unique_id]]

In [84]:
' 直近のRefusedされた月を結合し、その前後の集計を行う '
ref = abp.query("NAME_CONTRACT_STATUS=='Refused'")
ref_latest = ref.groupby(unique_id)['DAYS_DECISION'].max().reset_index().rename(columns={'DAYS_DECISION':'LATEST_REFUSE'})
apr_ref = apr.merge(ref_latest, on=unique_id, how='inner')

num_list = ['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY']
ref_bef = apr_ref.query('DAYS_DECISION < LATEST_REFUSE')[num_list+[unique_id]]
ref_aft = apr_ref.query('DAYS_DECISION >= LATEST_REFUSE')[num_list+[unique_id]]

In [114]:
# pos = pd.read_csv('../data/POS_CASH_balance.csv')
dpd_id = pos.query('SK_DPD>0')[unique_id].drop_duplicates().values

In [32]:
# one = apr.copy().query('DAYS_DECISION>=-6')
# two = apr.copy().query('DAYS_DECISION>=-24')
one = df.copy().query('row_no<=1')
two = df.copy().query('row_no<=5').query('row_no>1')
# one = apr.copy().set_index(unique_id).loc[dpd_id, :].query('DAYS_DECISION>=-1')
# two = apr.copy().set_index(unique_id).loc[dpd_id, :].query('DAYS_DECISION>=-3')

In [122]:
' 前後比の特徴量を作成する '
# prefix = 'abp_EMP_RATIO_'
prefix = 'abp_1_10num_RATIO_'
method_list = ['sum', 'mean', 'std', 'max', 'min']
# num_list = ['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY']
# num_list = ['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY', 'DAYS_DECISION', 'DAYS_DECISION_diff@', 'CREDIT_TERM']

after = one
before = two

for method in method_list:
    for num in num_list:
        aft = after.groupby(unique_id).agg({num:method}).reset_index()
        bef = before.groupby(unique_id).agg({num:method}).reset_index().rename(columns={num:f'{num}_bef'})
        ratio = aft.merge(bef, on=unique_id, how='inner')
        ratio[f'{num}_{method}@'] = ratio[num] / ratio[f'{num}_bef']
        
        result = base.merge(ratio, on=unique_id, how='left')
        make_npy(result=result, logger=logger, ignore_list=ignore_features, prefix=prefix)

# select_level_agg(base=base, data=df, ignore_list=ignore_features,  level=unique_id, method_list=method_list, num_list=num_list, prefix=prefix)

In [123]:
prefix = 'abp_'
month = apr[[unique_id, 'DAYS_DECISION']].drop_duplicates().query('DAYS_DECISION>=-12')
month['DAYS_DECISION'] = month['DAYS_DECISION'].astype('int')
month_size = month.groupby(unique_id)['DAYS_DECISION'].size().to_frame().rename(columns={'DAYS_DECISION':'month_variety@'})
month_std = month.groupby(unique_id)['DAYS_DECISION'].std().to_frame().rename(columns={'DAYS_DECISION':'month_std@'})
result = month_size.join(month_std)

result = base.merge(result, on=unique_id, how='left')
make_npy(result=result, logger=logger, ignore_list=ignore_features, prefix=prefix)

In [5]:
def extract_same_during_annuity(row):

    remaing = row_list.copy()
    remaing.remove(row)
    
    ''' 
    tmp_resultは、ループで回ってきたrow_noを持ったレコードに絞ったDF。
    この後のremaing_loopでこのレコードと同じ期間を持ったレコードを絞り、UNIONしていく 
    '''
    tmp_result = apr.query(f'row_no=={row}')[[unique_id, b_id, 'row_no']]
    tmp = apr.query(f'row_no=={row}')[[unique_id, b_id, 'row_no', 'DAYS_DECISION', 'DAYS_LAST_DUE']].rename(columns={'DAYS_DECISION':'start', 'DAYS_LAST_DUE':'end'})
    
    for rem in tqdm(remaing):
        tmp2 = apr.query(f'row_no=={rem}')[[unique_id, b_id, 'row_no', 'DAYS_DECISION', 'DAYS_LAST_DUE']].rename(columns={'DAYS_DECISION':'start2', 'DAYS_LAST_DUE':'end2', 'row_no':'same_term_row_no', b_id:'add_id'})
        
        ' rowの期間とremaingで回ってきた他の行の期間を比較する為、merge。 '
        df = tmp2.merge(tmp, on=unique_id, how='left')
        
        start = df['start'].values
        end = df['end'].values
        start2 = df['start2']
        end2 = df['end2']
        
        start2 = start2[start <= start2][start2 < end].to_frame().reset_index()['index'].to_frame()
        end2 = end2[start < end2][end2 <= end].to_frame().reset_index()['index'].to_frame()
        index = pd.concat([start2, end2], axis=0).drop_duplicates()
        index['include_flg'] = 1
        df.reset_index(inplace=True)
        same_term = df.merge(index, on='index', how='inner')[[unique_id, 'add_id']].rename(columns={'same_term_row_no':'row_no', 'add_id':b_id})
        
        same_term['row_no'] = rem
        tmp_result = pd.concat([tmp_result, same_term], axis=0)
        
    # ここでaprにmergeするのは、AMT_CREDITなどをつけるため
    target = apr[[unique_id, 'AMT_CREDIT', 'AMT_ANNUITY', 'row_no']].merge(tmp_result, on=[unique_id, 'row_no'], how='inner')
    target['serial_no'] = row
    print(row)
    return target

In [9]:
def pararell_same_during_annuity(base, df):
    '''
    同時期に支払いの生じていたbureau_idをまとめ、月あたり支払額の合計を作る。
    延滞を起こしたbureauについて、同時期に支払っていた金額を求める。また、それをINCOMEで割る。
    同時期に無理な割合の支払額を持っていたのではないか？という仮説
    '''
    
    ' DAYS_CREDIT_ENDDATEがNaNの場合、仮決めで3ヶ月後を完了日とする '
    df['DAYS_LAST_DUE'] = df['DAYS_LAST_DUE'].fillna(3)
    df.sort_values(by=[unique_id, 'DAYS_DECISION'], inplace=True)
    
    ' do_listにある行番号について、同時期の支払いを抽出する '
    p_list = pararell_process(extract_same_during_annuity, do_list)
    
    # 同時期のセットが3つとすると、それが3回ループでやってくるが、重み付き平均チックなのが良いので3つとも積んで平均とる
    print(len(p_list))
    result = pd.concat(p_list, axis=0)
    serial = result['serial_no'].min()
    
    result.sort_values(by=[unique_id, 'serial_no'], inplace=True)
    result['CNT_ANNUITY'] = result['AMT_CREDIT'] / result['AMT_ANNUITY']
    
    ' SUMについての処理 '
    same_sum = result.groupby([unique_id, 'serial_no'])[['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY']].sum()
    same_sum.columns = [f'same_{col}_sum' for col in same_sum.columns]
    same_max = same_sum.groupby(unique_id)['same_AMT_ANNUITY_sum'].max().reset_index()
    same_sum = same_sum.merge(same_max, on=[unique_id, 'same_AMT_ANNUITY_sum'], how='inner')
    
    same_std = result.groupby([unique_id, 'serial_no'])[['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY']].std()
    same_std.columns = [f'same_{col}_std' for col in same_std.columns]
    same_max = same_std.groupby(unique_id)['same_AMT_ANNUITY_std'].max().reset_index()
    same_std = same_std.merge(same_max, on=[unique_id, 'same_AMT_ANNUITY_std'], how='inner')
    
    same_sum.set_index(unique_id, inplace=True)
    same_std.set_index(unique_id, inplace=True)
    
    same = same_sum.join(same_std).drop_duplicates()
    
    same.to_csv(f'../data/csv_feature/{value}_same_term_feature_{serial}.csv')
    
    return
    
# ' 直近2年で見る '
# data = apr.copy().query('DAYS_DECISION>=-12')
# value = 'AMT_ANNUITY'

# ' do_listにある行番号について、同時期の支払いを抽出する。20以上になると並列処理のエラーがでる？ '
# do_list = list(range(21, 50, 1))
# ' 同時期の支払いかどうか確認するID別の最大行数 '
# row_list = list(data['row_no'].drop_duplicates().values)
# print(row_list)
# sys.exit()

# pararell_same_during_annuity(base, data)
# sys.exit()

' 結合 '
result = pd.DataFrame([])
for i in range(1, 30, 10):
    if i==21:continue
    tmp = pd.read_csv(f'../data/csv_feature/AMT_ANNUITY_same_term_feature_{i}.csv')
    
    if len(result)==0:
        result = tmp
    else:
        result = pd.concat([result,tmp], axis=0)

result = result.groupby(unique_id, as_index=False).max()
print(result.shape)
print(result.head())
result.to_csv(f'../data/csv_feature/AMT_ANNUITY_same_during_feature_1year_same.csv', index=False)

(266190, 7)
   SK_ID_CURR  same_AMT_CREDIT_sum  same_AMT_ANNUITY_sum  \
0      100001            279720.00             279720.00   
1      100002            406597.50              24700.50   
2      100003           1258845.93              89893.98   
3      100004             94537.80              94537.80   
4      100005            222768.00              17370.00   

   same_CNT_ANNUITY_sum  same_AMT_CREDIT_std  same_AMT_ANNUITY_std  \
0              1.000000        164428.445703           6688.431206   
1             16.461104         15337.146084           7680.997918   
2             28.007347        397090.612720              0.000000   
3              1.000000         52604.501880            750.802766   
4             12.824870                  NaN                   NaN   

   same_CNT_ANNUITY_std  
0             24.058214  
1              2.828427  
2              8.834643  
3             12.902546  
4                   NaN  


In [71]:
result = pd.read_csv(f'../data/csv_feature/AMT_ANNUITY_same_during_feature_2year_same_max.csv')
result = result.merge(app[[unique_id, 'AMT_INCOME_TOTAL', 'DAYS_BIRTH_log10']], on=unique_id, how='inner')

same_list = [col for col in result.columns if col.count('same')]

second = ['AMT_INCOME_TOTAL', 'DAYS_BIRTH_log10']
for num in same_list:
    result = division_feature(df=result, first=num, second=second[0])
result = base.merge(result, on=unique_id, how='left')

In [124]:
# for col in result.columns :
#     if col.count('same'):
#         result.rename(columns={col:f'{col}@'}, inplace=True)
prefix = 'abp_same_2year_'
make_npy(result=result, ignore_list=ignore_features, logger=logger, prefix=prefix)

In [89]:
# abp.head()
# abp.loc[ref_id, :].query('DAYS_DECISION<=-1')[unique_id].drop_duplicates().shape
# abp.loc[ref_id, :].query('DAYS_DECISION>=-1').query(f'DAYS_LAST_DUE == DAYS_LAST_DUE').query(f'DAYS_LAST_DUE <= 0').shape
# abp.loc[ref_id, :].query('DAYS_DECISION>=-12').query(f'DAYS_LAST_DUE != DAYS_LAST_DUE or DAYS_LAST_DUE > 0')[unique_id].drop_duplicates().shape
abp.loc[ref_id, :].query('DAYS_DECISION>=-6').query(f'DAYS_LAST_DUE == DAYS_LAST_DUE').query('DAYS_LAST_DUE <= 0')[unique_id].drop_duplicates().shape

(1090,)

In [204]:
' 延滞したことのあるID '
# pos = pd.read_csv('../data/POS_CASH_balance.csv')
dpd_id = list(pos.query('SK_DPD>0')[unique_id].drop_duplicates().values)
' RefusedされたことのあるID '
ref_id = list(abp.query("NAME_CONTRACT_STATUS=='Refused'")[unique_id].drop_duplicates().values)
' OVERDUEのID '
over_id = list(bureau.query('AMT_CREDIT_MAX_OVERDUE>0')[unique_id].drop_duplicates().values)

dpd_ref_id = list(set(dpd_id + ref_id))
dpd_over_id = list(set(dpd_id + over_id))
ref_over_id = list(set(ref_id + over_id))
dpd_ref_over_id = list(set(dpd_id + ref_id + over_id))

# np.save(arr=np.array(dpd_ref_over_id), file='../data/dpd_ref_over_id')

170546


SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [126]:
' 期間を絞り単純集計する '
prefix='abp_latest15_'
method_list = ['sum', 'mean', 'std', 'max', 'min']
# data = apr.copy().query('DAYS_DECISION>=-24')
# data = apr.copy().query('DAYS_DECISION>=-1').query(f'DAYS_LAST_DUE == DAYS_LAST_DUE').query(f'DAYS_LAST_DUE <= 0')
# data = abp.loc[ref_id, :].query('DAYS_DECISION>=-12').query(f'DAYS_LAST_DUE != DAYS_LAST_DUE or DAYS_LAST_DUE > 0')
# agg_id = dpd_ref_over_id
# data = abp.loc[agg_id, :].query('DAYS_DECISION>=-1').query(f'DAYS_LAST_DUE != DAYS_LAST_DUE or DAYS_LAST_DUE > 0')
# data = abp.loc[agg_id, :].query('DAYS_DECISION>=-12').query(f'DAYS_LAST_DUE == DAYS_LAST_DUE').query(f'DAYS_LAST_DUE <= 0')

'回数を絞り単純集計する'
data = df.copy().query('row_no<=15')

# num_list = ['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_ANNUITY', 'CREDIT_TERM', 'DAYS_LAST_DUE', 'DAYS_DECISION_diff@']
# data = data[[unique_id, 'AMT_INCOME_TOTAL', 'DAYS_DECISION']+num_list]

# for num in num_list:
#     if num.count('DAYS') or num.count('TERM'):continue
#     data = division_feature(df=data, first=num, second='AMT_INCOME_TOTAL')
    
# num_list = num_list + [col for col in data.columns if col.count('@') and col.count('AMT_INCOME') and col.count('div')]
select_level_agg(base=base, data=data, ignore_list=ignore_features,  level=unique_id, method_list=method_list, num_list=num_list, prefix=prefix)
# del data

In [125]:
' 重み付き平均 '
weight_list = [0.99, 0.97, 0.95]
feature_list = num_list
for num in feature_list:
    for weight in weight_list:
        wavg = exp_weight_avg(data=data, level=unique_id, weight=weight, label='DAYS_DECISION', value=num)
        result = base.merge(wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)